## **Given Task 2**

2. **Task Decomposition and Workflow Management**
 - Create a system that can break down complex tasks of your choice into smaller, manageable sub-tasks.
 - Implement a workflow management system that can assign sub-tasks to appropriate agents based on their capabilities.
 - Design and implement a simple UI, CLI/TUI or Gui of your choice for users to submit tasks and monitor progress. Ideally you would build a simple React UI, possibly using your agentic worker to create the code.

In [1]:
%%capture

!pip install -r requirements.txt

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
api_base_secret = os.getenv('api_base_secret')
azure_api_key_secret = os.getenv('azure_api_key_secret')

In [4]:
config_list = [{
    "engine": "gpt-35-turbo-16k", 
    "model": "gpt-35-turbo-16k",
    "api_base": api_base_secret, # Your Secret Base URL Endpoint :)
    "api_type": "azure", 
    "api_version": "2023-07-01-preview",
    "api_key": azure_api_key_secret # Your Secret Azure API key :)
}]

In [5]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent
import chromadb
import autogen
# from tqdm.autonotebook import tqdm, trange


llm_config = {
    "request_timeout": 120,
    "seed": 21,
    "config_list": config_list,
    "temperature": 0.1,
}

autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

boss = autogen.UserProxyAgent(
    name="Boss",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="The boss who ask questions and give tasks.",
    code_execution_config=False,  # we don't want to execute code in this case.
)

boss_aid = RetrieveUserProxyAgent(
    name="Boss_Assistant",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    retrieve_config={
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/facebook/react/main/README.md",
        "chunk_token_size": 1500,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="owais_react_developer/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

owais_coder = AssistantAgent(
    name="Senior_full_stack_developer_OWAIS",
    is_termination_msg=termination_msg,
    system_message="You are a senior Full Stack Developer. Your Name is Owais. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

pm = autogen.AssistantAgent(
    name="Product_Manager",
    is_termination_msg=termination_msg,
    system_message="You are a product manager. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

reviewer = autogen.AssistantAgent(
    name="Code_Reviewer",
    is_termination_msg=termination_msg,
    system_message="You are a code reviewer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

PROBLEM = "Design and implement a simple UI/Gui for users to submit tasks and monitor progress.\
           Ideally you would build a simple React UI. Give me sample code and ways to execute it in console terminal"

def _reset_agents():
    boss.reset()
    boss_aid.reset()
    owais_coder.reset()
    pm.reset()
    reviewer.reset()

In [6]:
def rag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss_aid, owais_coder, pm, reviewer], messages=[], max_round=30
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss_aid as this is the user proxy agent.
    boss_aid.initiate_chat(
        manager,
        problem=PROBLEM,
        n_results=5,)
    
    
rag_chat()

Trying to create collection.


/Users/owaisahmad/Desktop/Agentic Worker/env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/owaisahmad/Desktop/Agentic Worker/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


doc_ids:  [['doc_0']]
Adding doc_id doc_0 to context.
Boss_Assistant (to chat_manager):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Design and implement a simple UI/Gui for users to submit tasks and monitor progress.           Ideally you would build a simple React UI. Give me sample code and ways to execute it in console terminal

Context is: # [React](https://react.dev/) &middot; [![GitHub license](https://img.shields.io/badge/license-MIT-blue.svg)](https://github.com/facebook/react/blob/main/LICENSE) [![npm version](h

In [7]:
def norag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss, owais_coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,)
norag_chat()

Boss (to chat_manager):

Design and implement a simple UI/Gui for users to submit tasks and monitor progress.           Ideally you would build a simple React UI. Give me sample code and ways to execute it in console terminal

--------------------------------------------------------------------------------
Senior_full_stack_developer_OWAIS (to chat_manager):

Sure, I can help you with that. Here's a sample code for a simple React UI to submit tasks and monitor progress:

```jsx
import React, { useState } from 'react';

const App = () => {
  const [tasks, setTasks] = useState([]);
  const [newTask, setNewTask] = useState('');

  const handleTaskChange = (e) => {
    setNewTask(e.target.value);
  };

  const handleTaskSubmit = (e) => {
    e.preventDefault();
    setTasks([...tasks, newTask]);
    setNewTask('');
  };

  return (
    <div>
      <h1>Task Manager</h1>
      <form onSubmit={handleTaskSubmit}>
        <input type="text" value={newTask} onChange={handleTaskChange} />
       